# Imports

In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

# Loading data

In [22]:
df = pd.read_csv('items_titles_test.csv') 

In [23]:
df.head()

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde


# Removing Stopwords 

In [5]:
stop_words = stopwords.words("portuguese")
df["ITE_ITEM_TITLE"] = df["ITE_ITEM_TITLE"].apply(
    lambda x: " ".join([word for word in x.split() if word not in (stop_words)])
)


# Stemming

In [7]:
porter_stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)


df["ITE_ITEM_TITLE"] = df["ITE_ITEM_TITLE"].apply(stem_sentences)

# Bag of Words

In [9]:
vect = CountVectorizer(analyzer = 'word', ngram_range = (1,2), min_df = 0.002)
bow_matrix = vect.fit_transform(df["ITE_ITEM_TITLE"])

In [11]:
print("Cantidad de palabras a utilizar: ",len(vect.get_feature_names_out()))

Cantidad de palabras a utilizar:  686


# Matriz de similitud

**Usamos la distancia del coseno**

In [24]:
item_bow_similarity = cosine_similarity(bow_matrix, bow_matrix)

In [25]:
#Convertimos a dataframe
item_bow_similarity_df = pd.DataFrame(item_bow_similarity,
                                        columns=df['ITE_ITEM_TITLE'].values,
                                        index=df['ITE_ITEM_TITLE'].values)

In [27]:
#Convertimos a NaN el triangulo inferior
item_bow_similarity_df = (item_bow_similarity_df.where(np.triu(np.ones(item_bow_similarity_df.shape))
                                                       .astype(np.bool)
                                                      )
                         ).stack().reset_index()

In [29]:
df_final = item_bow_similarity_df.rename(columns={'level_0': 'ITE_ITEM_TITLE',
                                                  'level_1': 'ITE_ITEM_TITLE',
                                                  0: 'score'
                                                 }
                                        )

In [30]:
df_final.head()

,ITE_ITEM_TITLE,ITE_ITEM_TITLE,score
0,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Esporte Valente - Masculino Kids,1.000000
1,Tênis Olympikus Esporte Valente - Masculino Kids,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,0.000000
2,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Usthemp Slip-on Temático - Labrador 2,0.136083
3,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Casual Feminino Moleca Tecido Tie Dye,0.123091
4,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Star Baby Sapatinho Conforto + Brinde,0.182574


In [ ]:
df_final = df_final.sort_values(by='score', ascending=False)

,ITE_ITEM_TITLE,ITE_ITEM_TITLE,score
23378247,teni feminino sapato feminino casual sapateni ...,teni feminino sapato feminino casual sapateni ...,1.0
0,têni olympiku esport valent - masculino kid,têni olympiku esport valent - masculino kid,1.0
23312529,têni moleca slip on super confortável 5657101 ...,têni moleca slip on super confortável 5657101 ...,1.0
23210140,slip teni mickey personagen disney slip feminino,slip teni mickey personagen disney slip feminino,1.0
23224779,sneaker têni chunki feminino flatform em couro...,sneaker têni chunki feminino flatform em couro...,1.0
...,...,...,...
7796089,tv 26 lg 26le5300,sapatenisteni masculino polo plu sport origin jog,0.0
7796090,tv 26 lg 26le5300,têni redley / estampa floral,0.0
7796091,tv 26 lg 26le5300,têni masculino olympiku cristal marinho origin...,0.0
31333794,têni infantil de led rosa,bicicleta elétrica miami aro 26 retrô 350w 7.8...,0.0
